In [1]:
from pathlib import Path
from rnamodif.data_utils.data_paths import name_to_files

train_pos_files = [name_to_files['nia_2022_pos']['train']]
train_neg_files = [
    name_to_files['nia_2022_neg']['train'],
    # name_to_files['nia_2020_neg']['train'],
    # name_to_files['nia_neuron_hek']['train'],
    # name_to_files['nia_neuron_ctrl']['train'],
    # name_to_files['nia_neuron_tdp']['train'],
]

valid_exp_to_files_pos = {
    '5eu_2020_pos':name_to_files['nia_2020_pos']['test'], 
    'Nanoid_pos':name_to_files['nano_pos_1']['test'], 
    '5eu_2022_chr1_pos':name_to_files['nia_2022_pos']['test'],
    '5eu_2022_chr20_pos':name_to_files['nia_2022_pos']['validation']
    
}

valid_exp_to_files_neg = {
    'UNM_2020':name_to_files['nia_2020_neg']['test'], 
    'Nanoid_neg':name_to_files['nano_neg_1']['test'], 
    '5eu_2022_chr1_neg':name_to_files['nia_2022_neg']['test'],
    '5eu_2022_chr20_neg':name_to_files['nia_2022_neg']['validation'],
    
}

#positive-negative-name
valid_auroc_tuples = [
    ('5eu_2022_chr1_pos', '5eu_2022_chr1_neg', '2022'),
    ('5eu_2020_pos', 'UNM_2020', '2020'),
    ('Nanoid_pos', 'Nanoid_neg', 'Nanoid'),
    ('5eu_2022_chr20_pos', '5eu_2022_chr20_neg', '2022 valid'),
]

In [8]:
def f(a,b,c):
    print(b+c)
    
params = {
    'b':10,
    'a':1,
    'c':3,
}
    
# f(**params)
f(a=1,b=2,c=3)

5


In [ ]:
from rnamodif.model import RodanPretrained
from rnamodif.data_utils.dataloading_5eu import TrainingDatamodule
import pytorch_lightning as pl
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

log_steps = 1000
model = RodanPretrained(lr=1e-4, warmup_steps=3000, frozen_layers=0, 
                        gru_layers=1, gru_dropout=0.2, gru_hidden=32, logging_steps=log_steps)

dm = TrainingDatamodule(
    train_pos_files=train_pos_files,
    train_neg_files=train_neg_files,
    valid_exp_to_files_pos=valid_exp_to_files_pos,
    valid_exp_to_files_neg=valid_exp_to_files_neg,
    valid_auroc_tuples=valid_auroc_tuples,
    batch_size=64, 
    window=4096,
    per_dset_read_limit=250, 
    shuffle_valid=True,
    workers=8,
    skip=1000,
)

experiment_name = '2022_gru32_standard'
checkpoint_callback = ModelCheckpoint(
    dirpath=f"/home/jovyan/RNAModif/rnamodif/checkpoints_pl/{experiment_name}", 
    save_top_k=2, 
    monitor="valid_loss", 
    save_last=True, 
    save_weights_only=False
)

early_stopping_callback = EarlyStopping(monitor="2022 valid auroc MEAN", mode="max", patience=100)
logger = CometLogger(api_key="TEVQbgxxvilM1WdTyqZLJ57ac", project_name='RNAModif', experiment_name=experiment_name) 
trainer= pl.Trainer(
    max_steps = 1000000, logger=logger, accelerator='gpu',
    auto_lr_find=False, val_check_interval=log_steps,  
    log_every_n_steps=log_steps, benchmark=True, precision=16,
    callbacks=[checkpoint_callback, early_stopping_callback],
    # resume_from_checkpoint=f'/home/jovyan/RNAModif/rnamodif/checkpoints_pl/{experiment_name}/lastX.ckpt'
)


trainer.fit(model, dm)

/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
CometLogger will be initialized in online mode
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Generating valid dataset


 12%|█▎        | 1/8 [00:00<00:06,  1.06it/s]

5eu_2020_pos 250


 25%|██▌       | 2/8 [00:02<00:06,  1.06s/it]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]